In [1]:
import  torch
import  numpy as np
import  matplotlib.pyplot as plt
from    keras import datasets
import  import_ipynb

from RegretBound import opt_param
from data import MNIST_data
from model import OFL_Model
from utils import sigma_diff

K = 100        # Number of clients
D = 34826       # number of parameters (MNIST)
P = 0.1        # Com. overhead reduction rate from FedOGD
L = 4           # Local training time (Batch Size)

s, _, b, p = opt_param(P, D, show_result=False)
p2 = p * L
print(s, b, p, p2)

ModuleNotFoundError: No module named 'torch'

In [ ]:
x_train, y_train, input_size = MNIST_data(iid=True, shuffle=False)
task = 'clf'

Model_list = []

Model_list.append(OFL_Model('FedOGD', task, K, [False, 0, 0], 1, 1, input_size))
Model_list.append(OFL_Model('OFedAvg', task, K, [False, 0, 0], P, 1, input_size))
Model_list.append(OFL_Model('FedOMD', task, K, [False, 0, 0], 1, 10, input_size))
Model_list.append(OFL_Model('OFedIQ', task, K, [True, s, b], p, 1, input_size))

In [ ]:
i_max = len(y_train) // K
iter_max = 15
print(iter_max, i_max)

for model in Model_list:
    for iter in range(iter_max):
        for i in range(i_max):
            model.train(x_train[K*i : K*(i+1)], y_train[K*i : K*(i+1)], ((i_max * iter) + (i+1)) % model.L)
        print(iter, end =',')

result_list = []
for model in Model_list:
    result_list.append(model.pull_result())

In [ ]:
import pickle

code = input("Code: ")

for i, model in enumerate(Model_list):
    name = model.name
    with open(f"./result/{task}_{name}_{code}.pkl","wb") as f:
        pickle.dump(result_list[i], f)

In [ ]:
import pickle 

with open(f"./result/clf_FedOGD_sigma_iid.pkl", "rb") as f:
    r1 = pickle.load(f)
with open(f"./result/clf_FedOGD_sigma_niid.pkl", "rb") as f:
    r2 = pickle.load(f)

print(r1[250], r1[500])
print(r2[250], r2[500])